# 8 - Memory Management
##  GADK - The Standard Way: Using EventActions.state_delta for More Complicated Updates
- For times when you need to do more complex things – like updating several keys at once, saving things that aren't just text, targeting specific scopes like user: or app:, or making updates that aren't tied to the agent's final text reply – you'll manually build a dictionary of your state changes (the state_delta) and include it within the EventActions of the Event you're appending.
- Let's look at a code example demonstrating explicit state update via EventActions 


In [1]:
# Import necessary classes from the Google Agent Developer Kit (ADK)
# InMemorySessionService: A service for managing sessions in memory (for testing).
# Session: Represents a single conversation thread.
# Event: Represents an interaction or action within a session.
# EventActions: Holds actions associated with an Event, including state changes.
# Part, Content: Classes for representing message content.
import time  # Import the time module for timestamps
from google.adk.sessions import InMemorySessionService, Session
from google.adk.events import Event, EventActions
from google.genai.types import Part, Content

In [2]:
# --- Setup ---
# Initialize the InMemorySessionService.
session_service = InMemorySessionService()

# Define identifiers for the application, user, and session.
app_name, user_id, session_id = "state_app_manual", "user2", "session2"

# Create a new session using the session service.
# Initialize the session with some starting state data.
session = session_service.create_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id,
    state={"user:login_count": 0, "task_status": "idle"},  # Initial state dictionary
)

# Print the initial state of the session.
print(f"Initial state: {session.state}")

Initial state: {'user:login_count': 0, 'task_status': 'idle'}


In [3]:
# --- Define State Changes ---
# Get the current time to use for a timestamp in the state.
current_time = time.time()

# Define a dictionary containing the state changes you want to apply.
# This is the 'state_delta'.
state_changes = {
    "task_status": "active",  # Update an existing session state key
    # Read the current value from session.state and increment it.
    # Use .get() with a default (0) in case the key doesn't exist initially.
    "user:login_count": session.state.get("user:login_count", 0)
    + 1,  # Update a user-scoped state key
    "user:last_login_ts": current_time,  # Add a new user-scoped state key with a timestamp
    "temp:validation_needed": True,  # Add a temporary state key (often discarded by persistent services)
}

In [4]:
# --- Create Event with Actions ---
# Create an EventActions object and pass the state_changes dictionary to its state_delta parameter.
# This links the desired state updates to this specific event.
actions_with_update = EventActions(state_delta=state_changes)

# Create a new Event object.
# This event represents an action that occurred (e.g., a system event, a tool completing, etc.).
# It includes the actions object containing the state delta.
system_event = Event(
    # invocation_id="inv_login_update", # Example: An optional identifier for the invocation
    author="system",  # Specify the author of the event (e.g., 'system', 'agent', 'tool_name')
    actions=actions_with_update,  # Attach the actions object with the state delta
    timestamp=current_time,  # Include a timestamp for the event
    # content might be None or represent the action taken.
    # content=Content(parts=[Part(text="System updated state based on login event.")]) # Example of including content
)

In [5]:
# --- Append the Event (This updates the state) ---
# Append the created event to the session using the session service.
# When append_event is called, the SessionService processes the event's actions,
# including applying the state_delta to the session's state.
# Note: In a real ADK application, append_event is often asynchronous and would need 'await'.
# For this simplified synchronous example, we call it directly.
session_service.append_event(session, system_event)


# Print a confirmation message.
print("`append_event` called with explicit state delta.")

`append_event` called with explicit state delta.


In [6]:
# --- Check Updated State ---
# Retrieve the session again from the service to get its latest state.
updated_session = session_service.get_session(
    app_name=app_name, user_id=user_id, session_id=session_id
)

# Print the state after the event was appended.
# You should see the keys and values from state_changes reflected in the session's state.
print(f"State after event: {updated_session.state}")

# Expected output might look something like this (timestamp will vary):
# {'user:login_count': 1, 'task_status': 'active', 'user:last_login_ts': 1678886400.0, 'temp:validation_needed': True}
# Note: In a persistent SessionService, 'temp:' keys are typically not saved.
# In this InMemory mock, it might appear, but the concept is they are transient.

# Clean up the session (optional, but good practice in longer-running apps)
# session_service.delete_session(app_name, user_id, session_id)

State after event: {'user:login_count': 1, 'task_status': 'active', 'user:last_login_ts': 1747736091.932589}
